In [1]:
from modules.gem import GeM
import torch
import torch.nn.functional as F
import torch.nn as nn
from tqdm import tqdm
import numpy as np
from utils.train_util import set_seed
from torch.utils.data import DataLoader
from datasets.dl import GeMData, GeMClass
from datasets.config import GeMConfig, SwinConfig

2022-05-18 19:26:27.473937: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-05-18 19:26:27.473980: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
mi = GeMConfig('cifar100')
mi.set_arch('mvit')
model = GeM(mi)

load MViT CLS
load MViT trained parameters <All keys matched successfully>


In [3]:
# pic_matrix = torch.ByteTensor(np.load("data/imageset_small.npy"))
pic_matrix = torch.ByteTensor(np.random.randint(low=0, high=255, size=(256, 3, 224, 224)))
dataset_img = torch.LongTensor(np.random.randint(low=0, high=256, size=(1280, 1)))
dataset_label = torch.LongTensor(np.random.randint(low=0, high=100, size=(1280, 1)))

In [4]:
train_dataset = GeMClass(pic_matrix, torch.cat([dataset_img, dataset_label], dim=-1))
train_data_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)
# input_label = torch.zeros((32), dtype=torch.long).to(0)
input_label = torch.randint(low=0, high=100, size=(32, 1)).reshape(-1)

In [5]:
optimizer = torch.optim.AdamW(params=model.parameters(), lr=5e-4)

In [6]:
for epoch in range(2):
    model.train()
    model.zero_grad()
    index = 0
    steps_one_epoch = len(train_data_loader)
    enum_dataloader = tqdm(train_data_loader, total=steps_one_epoch, desc="EP-{} train".format(epoch))
    loss_list = []
    for data in enum_dataloader:
    #     if index >= steps_one_epoch:
    #         break

        model_in = data[:, :-1] / 255.0
#         pred = model.st(model_in.reshape(-1, 3, 224, 224))
#         print(pred.size())
        pred = model.predict_class(model_in, 224)
        loss = F.cross_entropy(pred, data[:, -1])
        loss_list.append(loss)

        loss.backward()
        optimizer.step()
        model.zero_grad()

        enum_dataloader.set_description("EP-{} train loss: {}".format(epoch, loss))
        enum_dataloader.refresh()
        index += 1
    
    print('epoch {} end'.format(epoch))


EP-0 train loss: 4.60898494720459:   2%|██▋                                                                                                        | 4/160 [00:18<12:06,  4.66s/it]


KeyboardInterrupt: 

In [9]:
for i, (pred, thw) in enumerate(preds):
    print(i, pred.size(), thw)

0 torch.Size([8, 3137, 96]) [1, 56, 56]
1 torch.Size([8, 3137, 192]) [1, 56, 56]
2 torch.Size([8, 785, 192]) [1, 28, 28]
3 torch.Size([8, 785, 384]) [1, 28, 28]
4 torch.Size([8, 197, 384]) [1, 14, 14]
5 torch.Size([8, 197, 384]) [1, 14, 14]
6 torch.Size([8, 197, 384]) [1, 14, 14]
7 torch.Size([8, 197, 384]) [1, 14, 14]
8 torch.Size([8, 197, 384]) [1, 14, 14]
9 torch.Size([8, 197, 384]) [1, 14, 14]
10 torch.Size([8, 197, 384]) [1, 14, 14]
11 torch.Size([8, 197, 384]) [1, 14, 14]
12 torch.Size([8, 197, 384]) [1, 14, 14]
13 torch.Size([8, 197, 384]) [1, 14, 14]
14 torch.Size([8, 197, 768]) [1, 14, 14]
15 torch.Size([8, 50, 768]) [1, 7, 7]
16 torch.Size([8, 50, 768]) [1, 7, 7]
